<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Onehot_A" data-toc-modified-id="Onehot_A-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Onehot_A</a></span></li><li><span><a href="#A_cat" data-toc-modified-id="A_cat-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>A_cat</a></span></li><li><span><a href="#A_hour" data-toc-modified-id="A_hour-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>A_hour</a></span></li><li><span><a href="#A_cat_xgb" data-toc-modified-id="A_cat_xgb-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>A_cat_xgb</a></span></li><li><span><a href="#B_cat_xgb" data-toc-modified-id="B_cat_xgb-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>B_cat_xgb</a></span></li><li><span><a href="#B_cat" data-toc-modified-id="B_cat-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>B_cat</a></span></li><li><span><a href="#A_his" data-toc-modified-id="A_his-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>A_his</a></span></li><li><span><a href="#结论" data-toc-modified-id="结论-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>结论</a></span></li><li><span><a href="#B_cat" data-toc-modified-id="B_cat-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>B_cat</a></span></li><li><span><a href="#合并" data-toc-modified-id="合并-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>合并</a></span></li><li><span><a href="#'C':-0.1" data-toc-modified-id="'C':-0.1-11"><span class="toc-item-num">11&nbsp;&nbsp;</span>'C': 0.1</a></span></li></ul></div>

# Onehot_A

In [6]:
# coding: utf-8

# filelist: 
#         train:40428967,  
#     minitrain:4042898,  
# miniminitrain:404291,  
#    test_click:4577464
#  

#import os
#os.chdir('/media/zhou/0004DD1700005FE8/AI/00/project_2/')
#os.chdir('E:/AI/00/project_2')

class flags(object):
    def __init__(self, file_name, onehot_cat):
        self.file_name = file_name
        self.onehot_cat = onehot_cat
        self.data_dir = '../data/project_2/data/{0}/'.format(self.onehot_cat)
        self.output_dir = '../data/project_2/output/{0}/'.format(self.onehot_cat)
        self.model_dir = '../data/project_2/models/{0}/'.format(self.onehot_cat)

class params(object):
    def __init__(self, onehot_name):
        self.threshold = 10
        self.chunksize = 1e3
        self.num_trees = 50
        self.deep = 15
        self.split = '='
        # ['A_cat', 'A_hour', 'A_xgb', 
        #  'B_cat', 'C_his']
        self.onehot_name = onehot_name
        self.lr_C = 1


In [7]:

import numpy as np
import pandas as pd
import time
import gc
import os
import scipy.sparse as ss
#from  sklearn.cross_validation  import  train_test_split 
#import xgboost as xgb
#from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

In [41]:
y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
merge_all = pd.DataFrame(index=np.arange(len(y_train)), columns=['click'], data=y_train)
merge_all[:10]

,click
0,0.0
1,1.0
2,0.0
3,0.0
4,1.0
5,0.0
6,0.0
7,0.0
8,0.0
9,0.0


# A_cat

In [42]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [43]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.1

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
#lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)

#拼接数据
merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
training . . . 
cost time:0


In [44]:
merge_all[:10]

,A_cat,click
0,0.184669,0.0
1,0.151759,1.0
2,0.028431,0.0
3,0.087226,0.0
4,0.186658,1.0
5,0.035379,0.0
6,0.297494,0.0
7,0.028510,0.0
8,0.278074,0.0
9,0.396243,0.0


# A_cat_xgb

In [45]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat_xgb')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [46]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
#lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)

#拼接数据
merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
training . . . 
cost time:0


# A_hour

In [47]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_hour')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [48]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
#lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)

#拼接数据
merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
training . . . 
cost time:0


In [49]:
merge_all[:10]

,A_hour,A_cat_xgb,A_cat,click
0,0.171241,0.193547,0.184669,0.0
1,0.164619,0.116275,0.151759,1.0
2,0.161582,0.055302,0.028431,0.0
3,0.146487,0.068892,0.087226,0.0
4,0.155272,0.159725,0.186658,1.0
5,0.168845,0.068892,0.035379,0.0
6,0.164455,0.259656,0.297494,0.0
7,0.153470,0.022967,0.028510,0.0
8,0.155573,0.313329,0.278074,0.0
9,0.155272,0.388862,0.396243,0.0


# B_cat

In [50]:
FLAGS = flags('minitrain', 'Onehot_B')
PARAMS = params('B_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [51]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.1

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
#lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)

#拼接数据
merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
training . . . 
cost time:0


In [52]:
merge_all[:10]

,B_cat,A_hour,A_cat_xgb,A_cat,click
0,0.169580,0.171241,0.193547,0.184669,0.0
1,0.169580,0.164619,0.116275,0.151759,1.0
2,0.169580,0.161582,0.055302,0.028431,0.0
3,0.169580,0.146487,0.068892,0.087226,0.0
4,0.169580,0.155272,0.159725,0.186658,1.0
5,0.169580,0.168845,0.068892,0.035379,0.0
6,0.169580,0.164455,0.259656,0.297494,0.0
7,0.159171,0.153470,0.022967,0.028510,0.0
8,0.169580,0.155573,0.313329,0.278074,0.0
9,0.169580,0.155272,0.388862,0.396243,0.0


# B_cat_xgb

In [53]:
FLAGS = flags('minitrain', 'Onehot_B')
PARAMS = params('B_cat_xgb')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [54]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
#lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)

#拼接数据
merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
training . . . 
cost time:0


In [55]:
merge_all[:10]

,B_cat_xgb,B_cat,A_hour,A_cat_xgb,A_cat,click
0,0.169566,0.169580,0.171241,0.193547,0.184669,0.0
1,0.169566,0.169580,0.164619,0.116275,0.151759,1.0
2,0.169566,0.169580,0.161582,0.055302,0.028431,0.0
3,0.169566,0.169580,0.146487,0.068892,0.087226,0.0
4,0.169566,0.169580,0.155272,0.159725,0.186658,1.0
5,0.169566,0.169580,0.168845,0.068892,0.035379,0.0
6,0.169566,0.169580,0.164455,0.259656,0.297494,0.0
7,0.168892,0.159171,0.153470,0.022967,0.028510,0.0
8,0.169566,0.169580,0.155573,0.313329,0.278074,0.0
9,0.169566,0.169580,0.155272,0.388862,0.396243,0.0


# A_his

In [56]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_his')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [57]:
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用

c=0.01

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
#lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)

#拼接数据
merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
training . . . 
cost time:0


In [58]:
merge_all[:10]

,A_his,B_cat_xgb,B_cat,A_hour,A_cat_xgb,A_cat,click
0,0.172588,0.169566,0.169580,0.171241,0.193547,0.184669,0.0
1,0.172588,0.169566,0.169580,0.164619,0.116275,0.151759,1.0
2,0.172588,0.169566,0.169580,0.161582,0.055302,0.028431,0.0
3,0.172588,0.169566,0.169580,0.146487,0.068892,0.087226,0.0
4,0.172588,0.169566,0.169580,0.155272,0.159725,0.186658,1.0
5,0.172588,0.169566,0.169580,0.168845,0.068892,0.035379,0.0
6,0.172588,0.169566,0.169580,0.164455,0.259656,0.297494,0.0
7,0.172588,0.168892,0.159171,0.153470,0.022967,0.028510,0.0
8,0.172588,0.169566,0.169580,0.155573,0.313329,0.278074,0.0
9,0.172588,0.169566,0.169580,0.155272,0.388862,0.396243,0.0


# A_cat

In [59]:
import xgboost as xgb

In [60]:
FLAGS = flags('minitrain', 'Onehot_A')
PARAMS = params('A_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [61]:
#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

bst_train = xgb.Booster(model_file=model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))

train_preds = bst_train.predict(xgtrain)
print('cost time:{0}'.format(int(time.time() - start_time)))

#拼接数据
merge_all.insert(loc=0, column=onehot_name+'_xgboost', value=train_preds)

Load Data
training . . . 
cost time:2


In [62]:
merge_all[:10]

,A_cat_xgboost,A_his,B_cat_xgb,B_cat,A_hour,A_cat_xgb,A_cat,click
0,0.162602,0.172588,0.169566,0.169580,0.171241,0.193547,0.184669,0.0
1,0.115767,0.172588,0.169566,0.169580,0.164619,0.116275,0.151759,1.0
2,0.036840,0.172588,0.169566,0.169580,0.161582,0.055302,0.028431,0.0
3,0.059306,0.172588,0.169566,0.169580,0.146487,0.068892,0.087226,0.0
4,0.165730,0.172588,0.169566,0.169580,0.155272,0.159725,0.186658,1.0
5,0.059306,0.172588,0.169566,0.169580,0.168845,0.068892,0.035379,0.0
6,0.212613,0.172588,0.169566,0.169580,0.164455,0.259656,0.297494,0.0
7,0.028032,0.172588,0.168892,0.159171,0.153470,0.022967,0.028510,0.0
8,0.300001,0.172588,0.169566,0.169580,0.155573,0.313329,0.278074,0.0
9,0.335770,0.172588,0.169566,0.169580,0.155272,0.388862,0.396243,0.0


# B_cat

In [63]:
FLAGS = flags('minitrain', 'Onehot_B')
PARAMS = params('B_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

In [64]:
#导入数据
print('Load Data')
X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
# 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
# 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
xgtrain = xgb.DMatrix(X_train, label = y_train,)

#设置参数, 开始训练
start_time = time.time()
print('training . . . ')

num_trees = 50
deep = 9

bst_train = xgb.Booster(model_file=model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))

train_preds = bst_train.predict(xgtrain,)
print('cost time:{0}'.format(int(time.time() - start_time)))

#拼接数据
merge_all.insert(loc=0, column=onehot_name+'_xgboost', value=train_preds)

Load Data
training . . . 
cost time:1


In [65]:
merge_all[:10]

,B_cat_xgboost,A_cat_xgboost,A_his,B_cat_xgb,B_cat,A_hour,A_cat_xgb,A_cat,click
0,0.169597,0.162602,0.172588,0.169566,0.169580,0.171241,0.193547,0.184669,0.0
1,0.169597,0.115767,0.172588,0.169566,0.169580,0.164619,0.116275,0.151759,1.0
2,0.169597,0.036840,0.172588,0.169566,0.169580,0.161582,0.055302,0.028431,0.0
3,0.169597,0.059306,0.172588,0.169566,0.169580,0.146487,0.068892,0.087226,0.0
4,0.169597,0.165730,0.172588,0.169566,0.169580,0.155272,0.159725,0.186658,1.0
5,0.169597,0.059306,0.172588,0.169566,0.169580,0.168845,0.068892,0.035379,0.0
6,0.169597,0.212613,0.172588,0.169566,0.169580,0.164455,0.259656,0.297494,0.0
7,0.168782,0.028032,0.172588,0.168892,0.159171,0.153470,0.022967,0.028510,0.0
8,0.169597,0.300001,0.172588,0.169566,0.169580,0.155573,0.313329,0.278074,0.0
9,0.169597,0.335770,0.172588,0.169566,0.169580,0.155272,0.388862,0.396243,0.0


# 合并

In [66]:
merge_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4042898 entries, 0 to 4042897
Data columns (total 9 columns):
B_cat_xgboost    float32
A_cat_xgboost    float32
A_his            float64
B_cat_xgb        float64
B_cat            float64
A_hour           float64
A_cat_xgb        float64
A_cat            float64
click            float32
dtypes: float32(3), float64(6)
memory usage: 262.2 MB


In [68]:
X_train = merge_all.iloc[:,:-1]
y_train = merge_all.iloc[:,-1]

In [70]:
#需要调优的参数
# 请尝试将L1正则和L2正则分开，并配合合适的优化求解算法（slover）
#tuned_parameters = {'penalty':['l1','l2'],
#                   'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000]
#                   }
solvers = ['sag','saga']
Cs = [ 0.01, 0.1, 1, 10, 100, ]
tuned_parameters = dict(solver = solvers, C = Cs)

lr_penalty= LogisticRegression()
grid= GridSearchCV(lr_penalty, tuned_parameters,cv=3, scoring='neg_log_loss')

In [71]:
grid.fit(X_train,y_train)
grid.cv_results_

/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zhou/anaconda3/lib/python3.5/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/zhou/anaconda3/lib/python3.5/site

{'mean_fit_time': array([ 23.87565573,  31.66397381,  27.03615912,  27.17015958,
         36.75948723,  30.29519407,  39.3010269 ,  52.46356757,
         42.15218798,  62.94082761]),
 'mean_score_time': array([ 0.60379656,  0.61651738,  0.62121439,  0.62269958,  0.6039269 ,
         0.61569357,  0.62351576,  0.59850152,  0.59500376,  0.59340556]),
 'mean_test_score': array([-0.39149532, -0.39149278, -0.38927864, -0.38928104, -0.38909571,
        -0.38909964, -0.38909956, -0.3891011 , -0.38910116, -0.38910243]),
 'mean_train_score': array([-0.3912609 , -0.39126094, -0.38899101, -0.38899091, -0.38879333,
        -0.3887933 , -0.38878945, -0.38878942, -0.38878933, -0.38878932]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 1 1 10 10 100 100],
              mask = [False False False False False False False False False False],
        fill_value = ?),
 'param_solver': masked_array(data = ['sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga' 'sag' 'saga'],
              mask = [False Fa

In [72]:
print(-grid.best_score_)
print(grid.best_params_)

0.389095711352
{'C': 1, 'solver': 'sag'}


In [73]:
onehot_name = 'all'
c=1

#导入模型

lr= LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('training . . . ')
lr.fit(X_train, y_train, )
print('cost time:{0}'.format(int(time.time() - start_time)))

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
train_predictions = np.around(train_preds)

train_accuracy = accuracy_score(y_train, train_predictions)
print ("Train Accuary: %.2f%%" % (train_accuracy * 100.0))
train_log_loss = log_loss(y_train, train_preds)
print ("Train log_loss: " , train_log_loss)


#保存模型
joblib.dump(lr, model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))


training . . . 
cost time:58
Train Accuary: 83.43%
Train log_loss:  0.388839094251


['../data/project_2/models/Onehot_B/LR_C1_all.sklearn']

# 'C': 0.1

In [25]:
c=0.1
#保存模型
joblib.dump(lr, model_path+'LR_C{0}.sklearn_more{1}__'.format(c,threshold))


['../data/project_2/models/Onehot_A/LR_C0.1.sklearn_more10__']

In [ ]:
lr_model = joblib.load(model_path+model_path+'LR_C{0}.sklearn'.format(c))